# Prediction of protein-protein interfaces
## Stanislav Belyasov, Minsk, 2021

In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"

In [2]:
from Bio.PDB import *
from Bio import SeqIO
from Bio.Blast import NCBIWWW, NCBIXML
from Bio.Align import AlignInfo
import Bio.pairwise2
from Bio.SeqUtils.ProtParam import ProteinAnalysis
from copy import deepcopy
import glob
import numpy as np
# import psaia
import pickle
from scipy.spatial.distance import cdist
import sys
from time import time
import itertools
import random

### Auxiliary functions

In [9]:
def read_pdb(fname: str, name: str = None):
    if name is None:
        _, name, _ = get_file_parts(fname)
    stx = PDBParser().get_structure(name, fname)
    assert len(stx) == 1      
    residues = Selection.unfold_entities(stx, 'R')
    pp = PPBuilder().build_peptides(stx)
    seq = ''.join([str(p.get_sequence()) for p in pp])
    Rdict = dict(zip(residues, range(len(residues))))
    seq2residue_i = [Rdict[r] for p in pp for r in p]
    return stx, residues, pp, seq, seq2residue_i


def get_file_parts(fname: str):
    path, name = os.path.split(fname)
    n = os.path.splitext(name)[0]
    ext = os.path.splitext(name)[1]
    return path, n, ext


def get_residue_i_id(fid: str):
    _, _, cid, (_, ridx, rinum) = fid
    return cid, str(ridx) + rinum.strip()


def get_coordinates(residue):
    coordinates = []
    for idx, r in enumerate(residue):   
        v = [ak.get_coord() for ak in r.get_list()]   
        coordinates.append(v)
    
    return coordinates

def get_distance(C0, C1, thr: float = np.inf):
    N0 = []
    N1 = []
    for i in range(len(C0)):
        for j in range(len(C1)):
            d = cdist(C0[i], C1[j]).min()
            if(d < thr):
                N0.append((i, j, d))
                N1.append((j, i, d))
    return N0, N1




### psiblast and PSSM functions

In [10]:
def run_psiblast(file: str, db: str = 'nr', ofile: str = None, niter: int = 3) -> int:
    if ofile is None:
        ofile = file

    print(os.getcwd())
    cmdstr = f'psiblast -query {file} -db {db} -out {ofile}.psi.txt -num_iterations {niter}' \
             f' -out_pssm {ofile}.pssm -out_ascii_pssm {ofile}.mat_'
    if os.system(cmdstr) == 0:
        print('PSIBLAST Successful : ', cmdstr)
        correct_file(f'{ofile}.mat_', f'{ofile}.mat')
        os.remove(f'{ofile}.mat_')
        return 0
    else:
        print('PSIBLAST Processing Failed.')
        return -1
    
def correct_file(ifile: str, ofile: str):
    with open(ofile, 'w') as of:
        for file in open(ifile, 'r'):
            fs = file.split()
            if len(fs) and fs[0].isdigit():
                if len(fs) < 44:
                    fpssm = ' '.join(split_str_into_len(file[9:69], l=3))
                    file = file[:9] + fpssm + file[69:]
            of.write(file)


def split_str_into_len(s: str, l: int = 3):
    return [(s[i: i + l]).strip() for i in range(0, len(s), l)]


def parse_pssm_file(fname: str):
    pssm = []
    psfm = []
    info = []

    with open(fname, 'r') as file:
        for tmp in file:
            tmp = tmp.split()
            if len(tmp) and tmp[0].isdigit():
                _pssm = [float(i) for i in tmp[2:22]]
                pssm.append(_pssm)
                _psfm = [float(i) / 100.0 for i in tmp[22:42]]
                psfm.append(_psfm)
                info.append(float(tmp[42]))

    return np.array(pssm).T, np.array(psfm).T, np.array(info)


def get_wpssm(xpm: np.array, HWS: int = 5):
    HWS = int(HWS)
    d, N = xpm.shape
    pm = np.hstack((np.zeros((d, HWS)), xpm, np.zeros((d, HWS))))
    ws = 2 * HWS + 1
    wpm = np.zeros((ws * d, N))
    for i in range(N):
        wpm[:, i] = pm[:, i:i + ws].flatten('F')
    return wpm

### Defining MyPBD object for .pdb processing

In [11]:
class MyPDB:
    def __init__(self, file_name: str, profile_dir: str = None):
        self.HWS_PSSM: float = 5.0

        distance_path, name, file_type = get_file_parts(file_name)
        if profile_dir is None:
            profile_dir = distance_path

        self.file_type: str = file_type.lower()
        self.name: str = name
        self.input_file_path: str = file_name
        self.chain_to_residue = dict()
        self.sg: float = -100.0
        self.threshold: float = -100
#         self.S: tp.Any = None
#         self.CN: tp.Any = None
        self.psaiaf = dict()

        self.stx, self.residues, _, self.seq, self.seq2residue_i = read_pdb(file_name)
        for idx, r in enumerate(self.residues):
            self.chain_to_residue[get_residue_i_id(r.get_full_id())] = idx

        self.__assign_pssm_dir(profile_dir)

    def __assign_pssm_dir(self, pssm_path: str):
        self.__get_psaia()
        self.__get_pssm(pssm_path)

    def save_to_fasta(self, file_path: str, save_header: bool = False):
        with open(file_path, 'w+') as f:
            if save_header:
                f.write(f'> {self.name}\n')
            f.write(self.seq)

    def __get_pssm(self, path: str):
        pssm_path = os.path.join(path, self.name + '.mat')
        if not (os.path.exists(pssm_path) and os.path.isfile(pssm_path)):
            file_path = os.path.join(path, self.name + '.fasta')
            self.save_to_fasta(file_path)
            run_psiblast(file_path, db='nr', ofile=path + self.name, niter=3)

        pssm_file = parse_pssm_file(pssm_path)
        N: int = len(self.residues)

        pssm, psfm, info = pssm_file
        wpssm = get_wpssm(pssm, self.HWS_PSSM)
        wpsfm = get_wpssm(psfm, self.HWS_PSSM)

        self.pssm = np.zeros((20, N))
        self.psfm = np.zeros((20, N))
        self.wpssm = np.zeros((wpssm.shape[0], N))
        self.wpsfm = np.zeros((wpsfm.shape[0], N))
        self.asa = np.zeros(N)
        self.asa.fill(np.nan)
        self.rasa = np.zeros(N)
        self.rasa.fill(np.nan)
        self.info = np.zeros(N)
        self.info.fill(np.nan)

        for k in range(len(self.seq)):
            idx = self.seq2residue_i[k]
            self.pssm[:, idx] = pssm[:, k]
            self.psfm[:, idx] = psfm[:, k]
            self.wpssm[:, idx] = wpssm[:, k]
            self.wpsfm[:, idx] = wpsfm[:, k]
            self.info[idx] = info[k]

    def save(self, output: str = None, base_dir: str = './'):
        if output is None:
            output = base_dir + self.name + '.pdb.pkl'
        with open(output, 'wb') as output:
            pickle.dump(self, output, -1)

    @classmethod
    def loader(cls, pickle_file):
        return pickle.load(open(pickle_file, "rb"))

    def __get_psaia(self):
        pdict = psaia.run_psaia(self.input_file_path)
        N = len(self.residues)
        self.psaiaf['casa'] = np.zeros((5, N))
        self.psaiaf['rasa'] = np.zeros((5, N))
        self.psaiaf['rrasa'] = np.zeros((5, N))
        self.psaiaf['rdpx'] = np.zeros((6, N))
        self.psaiaf['rcx'] = np.zeros((6, N))
        self.psaiaf['rhph'] = np.zeros(N)
        for idx, r in enumerate(self.residues):
            key = get_residue_i_id(r.get_full_id())
            if key in pdict.keys():
                casa, rasa, rrasa, rdpx, rcx, rhph = pdict[key]
                self.psaiaf['casa'][:, idx] = list(casa)
                self.psaiaf['rasa'][:, idx] = list(rasa)
                self.psaiaf['rrasa'][:, idx] = list(rrasa)
                self.psaiaf['rdpx'][:, idx] = list(rdpx)
                self.psaiaf['rcx'][:, idx] = list(rcx)
                self.psaiaf['rhph'][idx] = rhph
            else:
                print('key not found in PSAIA processing!')
                break


###  Getting feature files

In [7]:
import warnings
warnings.filterwarnings('ignore')

In [201]:
banned = ['1E6E', '1EWY', '1F34', '1RKE', '1GXD', '1GCQ', '1JK9']

In [9]:
import time

In [8]:
pdb_path = './data/targets/'
profile_path = pdb_path + 'profile/'
pdbpklpath = './data/pdbpkl/'

files = glob.glob(pdb_path + '*.pdb')
ban = False
for file in files:
    for b in banned:
        if b in file:
            ban = True
            break
    if ban:
        ban = False
        continue
    L = MyPDB(file, profile_path)
    L.save(base_dir=pdbpklpath)
    print("Yeap")
    time.sleep(1)

PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/2OUL_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/2OUL_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1R8S_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1R8S_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1QA9_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1QA9_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1CGI_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1CGI_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1JIW_l_u.pdb
Yeap
P

PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1BVN_r_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1FC2_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1ACB_r_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1ACB_r_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1OYV_r_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1OYV_r_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1LFD_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1LFD_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1FQJ_r_b.pdb
Yeap
P

PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1FFW_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1FFW_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1GLA_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1GLA_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1R6Q_l_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1R6Q_l_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1H1V_r_b.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1H1V_r_u.pdb
Yeap
PSAIA Successful : python3 /home/parallels/Downloads/course_project/script.py ./data/targets/1HE1_l_u.pdb
Yeap
P

### Getting positive and negative examples

In [12]:
class GetExamples:
    def __init__(self, idir: str, thr: float = 6.0):
        self.thr = thr
        self.dir = idir
        
        ufiles=glob.glob(os.path.join(idir, "*_l_u.pdb"))
        
        self.pos_ex = dict()
        self.neg_ex = dict()
        self.Plr = dict()
        
        ban = False
        
        for f in ufiles:
            for b in banned:
                if b in f:
                    ban = True
                    break
            if ban:
                ban = False
                continue
            _, cname, _ = get_file_parts(f)
            cname = cname[:-4]
            print(f"############################ Processing: {cname}")
            try:
                luname = os.path.join(idir, f'{cname}_l_u.pdb')
                runame = os.path.join(idir, f'{cname}_r_u.pdb')
                lbname = os.path.join(idir, f'{cname}_l_b.pdb')
                rbname = os.path.join(idir, f'{cname}_r_b.pdb')
                C = MyPDBComplex([lbname, rbname], thr)
                try:
                    D, LenU = C.find_nearest_for_unbound([luname,runame])            
                except Exception as e:
                    print(f'exception HERE SUKA: {str(e)}')
                D = D[0][1]
                lenL = LenU[0]
                lenR = LenU[1]
                P = []
                Pl = set([])
                Pr = set([])
                for i, j, d in D:
                    if ~(np.isnan(i) or np.isnan(j)):
                        Pl.add(i)
                        Pr.add(j)
                        P.append((i, j))                
            except:
                print(f"!!!!!!!!!!!!!!!!!!!!!!!! Error processing: {cname}")
                continue
            self.neg_ex[cname] = []
            self.pos_ex[cname] = (P, lenL, lenR)
            self.Plr[cname] = [Pl, Pr]
            
    def save(self, ofname: str = None):
        if ofname is None:
            ofname = f'E_{self.thr}.lbl.pkl'
        if pdbpklpath not in ofname:
            ofname = pdbpklpath + ofname
        print(f"File Saved: {ofname}")
        with open(ofname, 'wb') as output:
            pickle.dump(self, output)
    
    @classmethod   
    def loader(self, pklfile: str):
        return pickle.load(open(pklfile, "rb"))
    
    def get_neg_examples(self, cname: str):
        try:
            L = [range(self.pos_ex[cname][1]), range(self.pos_ex[cname][2])]
            X = set(itertools.product(*L))
            return X.difference(set(self.pos_ex[cname][0]))
        except:
            return None
    
    def select_neg_examples(self, cname: str = None, p: float = 100.0, M: int = 1000):    
        if cname is None:
            cname = self.pos_ex.keys()
        if M is None:
            M = np.inf            
        for c in cname:            
            if M == '+-':
                Pl, Pr = self.Plr[c]
                Nl = range(self.pos_ex[c][1])
                Nr = range(self.pos_ex[c][2])
                XPlPr = set(itertools.product(Pl, Pr)).difference(set(self.pos_ex[c][0]))
                XPlNr = (set(itertools.product(Pl, Nr)).difference(set(self.pos_ex[c][0]))).difference(XPlPr)
                XNlPr = (set(itertools.product(Nl, Pr)).difference(set(self.pos_ex[c][0]))).difference(XPlPr)
                XPN = XPlNr.union(XNlPr)
                
                npx = int(p * len(self.pos_ex[c][0]) / 100.0)
                npp = int(min(0.2 * npx, len(XPlPr)))
                N = set(random.sample(XPlPr, npp))
                npn = int(min(0.3 * npx, len(XPN)))
                N = N.union(set(random.sample(XPN, npn)))
                nn = npx - len(N)
                N = N.union(set(random.sample(XPN, nn)))
                self.neg_ex[c] = list(N)                        
            else:
                N = self.get_neg_examples(c)
                if M == '+':
                    n = int(min(p * len(self.pos_ex[c][0]) / 100.0, len(N)))
                else:
                    n = int(min(np.floor(p * len(N) / 100.0), M))
                self.neg_ex[c] = random.sample(N, n)

### MyPDBComplex class

In [13]:
def map_unbound_to_bound(u_seq, u_seq_2_res_i, ul_res, b_seq, b_seq_2_res_i, bl_res):
    u2b = [np.nan for k in range(ul_res)]
    b2u = [np.nan for k in range(bl_res)]
    aln = Bio.pairwise2.align.globalxs(u_seq, b_seq, -1, -0.1)[0]
    ui = 0
    bi = 0
    for k in range(len(aln[0])):
        uc = aln[0][k]
        bc = aln[1][k]
        if uc == bc:
            u2b[u_seq_2_res_i[ui]] = b_seq_2_res_i[bi]
            b2u[b_seq_2_res_i[bi]] = u_seq_2_res_i[ui]
        ui = ui + (uc != '-')
        bi = bi + (bc != '-')
    pu2b = sum(np.isnan(u2b)) / float(len(u2b))
    pb2u = sum(np.isnan(b2u)) / float(len(b2u))
    
    return u2b, b2u

In [14]:
class MyPDBComplex:
    def __init__(self, filenames, thr: float = 6.0):
        self.filenames = filenames
        self.N = len(filenames)
        self.thr = thr
        
        self.coordinates = []
        self.residues = []
        self.distances = [[[] for _ in range(self.N)] for _ in range(self.N)]
        self.seq = []
        self.seq_2_residue_i = []
        for i, f in enumerate(filenames):
            _, residue, _, seq, seq_2_res_i = read_pdb(f) 
            self.seq.append(seq)
            self.residues.append(residue)
            self.seq_2_residue_i.append(seq_2_res_i)
            self.coordinates.append(get_coordinates(residue))            
            for j in range(0, i):
                print(f'{j}, {i}')
                self.distances[i][j], self.distances[j][i] = \
                get_distance(self.coordinates[i], self.coordinates[j], thr)

    def find_nearest_for_unbound(self, ufnames):
        distances = [[[] for _ in range(self.N)] for _ in range(self.N)]
        unbound_to_bound = []
        bound_to_unbound = []
        unbound_len = []
        print(ufnames)
        for idx, f in enumerate(ufnames):
            _, u_residue, _, u_seq, u_seq_2_res_i = read_pdb(f)
            b_residue, b_seq, b_seq_2_res_i = self.residues[idx], self.seq[idx], self.seq_2_residue_i[idx]
            u2b, b2u = map_unbound_to_bound(u_seq, u_seq_2_res_i, len(u_residue), b_seq, b_seq_2_res_i, len(b_residue))
            unbound_len.append(len(u_residue))
            unbound_to_bound.append(u2b)
            bound_to_unbound.append(b2u)

        for i in range(self.N):
            for j in range(i + 1, self.N):
                oD = self.distances[i][j]
                for k in range(len(oD)):
                    try:
                        ov = oD[k]
                        vij = (bound_to_unbound[i][ov[0]], bound_to_unbound[j][ov[1]], ov[2])
                        vji = (bound_to_unbound[j][ov[1]], bound_to_unbound[i][ov[0]], ov[2])
                    except Exception as e:
                        print(f'in b2u and u2b')
                    distances[i][j].append(vij)
                    distances[j][i].append(vji)
        return distances, unbound_len

### Getting labels

In [203]:
pdbpath = './targets/'
E = GetExamples(pdbpath)
E.select_neg_examples(p=150.0, M='+-') 
E.save('EPN6_200.lbl.pkl')

############################ Processing: 1KXQ
0, 1
['./targets/1KXQ_l_u.pdb', './targets/1KXQ_r_u.pdb']
############################ Processing: 1GRN
0, 1
['./targets/1GRN_l_u.pdb', './targets/1GRN_r_u.pdb']
############################ Processing: 1FLE
0, 1
['./targets/1FLE_l_u.pdb', './targets/1FLE_r_u.pdb']
############################ Processing: 1R0R
0, 1
['./targets/1R0R_l_u.pdb', './targets/1R0R_r_u.pdb']
############################ Processing: 1AY7
0, 1
['./targets/1AY7_l_u.pdb', './targets/1AY7_r_u.pdb']
############################ Processing: 1GL1
0, 1
['./targets/1GL1_l_u.pdb', './targets/1GL1_r_u.pdb']
############################ Processing: 1J2J
0, 1
['./targets/1J2J_l_u.pdb', './targets/1J2J_r_u.pdb']
############################ Processing: 1IBR
0, 1
['./targets/1IBR_l_u.pdb', './targets/1IBR_r_u.pdb']
############################ Processing: 1D6R
0, 1
['./targets/1D6R_l_u.pdb', './targets/1D6R_r_u.pdb']
############################ Processing: 1B6C
0, 1
['./targets/1

### Building protein kernel

In [15]:
def process_psaia(pdb: MyPDB):
    RDPX_M = np.array([7.5131, 2.4013, 7.658, 1.8651, 8.0278, 7.0175])
    RDPX_m = -1.0
    RCX_M = np.array([11.153, 4.8229, 12.212, 4.4148, 19.84, 9.4199])
    RCX_m = -1.0
    RRASA_M = np.array([167.0, 368.04, 124.66, 316.96, 253.85])
    RRASA_m = 0.0
    CASA_m = np.array([1412.0, 235.32, 1174.2, 362.0, 940.64])
    CASA_M = np.array([36661.0, 7584.9, 29756.0, 15550.0, 21489.0])
    RASA_m = 0.0
    RASA_M = np.array([273.22, 134.51, 216.4, 173.26, 185.47])
    RHPH_m = -4.5
    RHPH_M = 4.5
    
    rdpx = ((pdb.psaiaf['rdpx'] - RDPX_m) / (RDPX_M - RDPX_m)[:, np.newaxis])
    rcx = ((pdb.psaiaf['rcx'] - RCX_m) / (RCX_M - RCX_m)[:, np.newaxis])
    rrasa = (pdb.psaiaf['rrasa'] - RRASA_m) / (RRASA_M - RRASA_m)[:, np.newaxis]
    casa = (pdb.psaiaf['casa'] - CASA_m[:, np.newaxis]) / (CASA_M - CASA_m)[:, np.newaxis]
    rasa = (pdb.psaiaf['rasa'] - RASA_m) / (RASA_M - RASA_m)[:, np.newaxis]
    fv = np.vstack((rdpx, rcx, rrasa, casa, rasa))
    return fv

def process_pssm(pssm):
    return pssm / (1e-10 + np.sum(pssm ** 2, axis=0) ** 0.5)

In [16]:
class ProteinKernel:
    def __init__(self, L, R, Lidx=None, Ridx=None):
        self.L = L
        self.R = R
        self.Lidx = Lidx
        self.Ridx = Ridx
        if Lidx is None:
            Lidx = range(len(L.R))
        if Ridx is None:
            Ridx = range(len(R.R))
            
        Kpssm = np.exp(-0.5 * cdist(process_pssm(L.wpssm).T, process_pssm(R.wpssm).T))
        Kpsfm = np.exp(-0.5 * cdist(process_pssm(L.wpsfm).T, process_pssm(R.wpsfm).T))
        Kpsaia = np.exp(-1.0 * (cdist(process_psaia(L).T, process_psaia(R).T) ** 2))
        Kin = Kpssm + Kpsfm + Kpsaia
        self.Lidx = np.array(self.Lidx)
        self.Lidx = self.Lidx[self.Lidx < len(Kin)]
        self.Ridx = np.array(self.Ridx)
        self.Ridx = self.Ridx[self.Ridx < len(Kin[0])]
        self.K = Kin[np.ix_(self.Lidx, self.Ridx)]
        
    def save(self, ofname: str):
        with open(ofname, 'wb') as output:
            pickle.dump(self, output)

    @classmethod   
    def loader(self, pklfile: str):
        return pickle.load(open(pklfile, "rb"))

### Creating common dataset

In [17]:
def copy_dict(d, *keys):
    return {key: d[key] for key in keys}


def get_valid_e(E0: GetExamples, pdbpklpath: str, cids = None):
    E = deepcopy(E0)
    K = set(E.pos_ex.keys())
    s = set()
    for k in K:
        f = os.path.join(pdbpklpath, k + "*.pdb.pkl")
        if len(glob.glob(f)) == 4:
            s.add(k)
    if cids is not None:
        s = s.intersection(set(cids))
#     s = s - {'1BUH', '1QA9', '1PPE', '1BVN', '1IRA', '1HE1', '1EFN'}
    tlist = list(s)
    E.pos_ex = copy_dict(E.pos_ex, *tlist)
    E.neg_ex = copy_dict(E.neg_ex, *tlist)
    return E

In [18]:
class db_kernel:
    def __init__(self, pdbpklpath: str, E0, ofname: str = None, incids=None):      
        E = get_valid_e(E0, pdbpklpath, incids) 
        self.E = E
        KK = list(E.pos_ex.keys())
        print(f"Number of Examples (+,-): ({np.sum([len(E.pos_ex[k][0]) for k in E.pos_ex])}," +
              f"{np.sum([len(E.neg_ex[k]) for k in E.neg_ex])}")
        txx0 = time.time()
        ignored = list(set(E0.pos_ex.keys()).difference(KK))
        if len(ignored) != 0:
            print(f'Ignoring the following complexes because at least one of the'
                  f'four required pdbpkl files could not be found in {pdbpklpath}, {len(ignored)}'
                  f'ignored')
        print(f"Evaluating kernel over: {KK}")
        kernel_dict = compute_kernel_dict(KK, pdbpklpath, E)       
        self.__db_kernel_to_pairwise_kernel__(KK, E, kernel_dict)
        print(f"TIME TAKEN FOR KERNEL EVALUATION (s): {time.time() - txx0}")
        txx0 = time.time()
        if ofname is None:
            ofname='dbdKernel.dbK.pkl'                   
        self.save(ofname)
        print(f"TIME TAKEN FOR Saving kernel file (s): {time.time() - txx0}")

            
    def __db_kernel_to_pairwise_kernel__(self, cids, E, kernel_dict):        
        S = [(cid, e, 1) for cid in cids for e in E.pos_ex[cid][0]]
        N = [(cid, e, -1) for cid in cids for e in E.neg_ex[cid]]
        S.extend(N)
        K, S, _ = db_kernel_to_pairwise_kernel(kernel_dict, S)    
        nanidx = ~np.isnan(K.diagonal())
        K = K[nanidx, :]
        K = K[:, nanidx]
        S = [s for s, n in zip(S, nanidx) if n]
        dK = np.diag(K)
        K = K / np.sqrt(dK[:, np.newaxis] * dK[:, np.newaxis].T)
        self.K = K
        self.S = S
        self.dK = dK

    def save(self, ofname: str = None):
        if ofname is None:
            ofname = 'dbdKernel.dbK.pkl'
        with open(ofname, 'wb') as output:
            pickle.dump((self.S, self.dK, self.E), output, -1)        
        with open(ofname + '.np', 'wb') as output:
            np.save(output, self.K)
        print(f"File Saved: {ofname}")
        
    @classmethod  
    def loader(self, pklfile: str):
        z = pickle.load(open(pklfile, "rb"))
        if len(z) == 3:
            K = np.load(pklfile + '.np')
            z = (K,) + z
        return z

In [19]:
def compute_kernel_dict(kk: list, pdbpklpath: str, examples = None, scid = None):
    aA: bool = False 
    if len(kk) == 0:
        assert scid is not None
        kk = [scid]
        aA = True
    Nc = len(kk)     
    bA = False
    if scid is None:
        clist = [(a, b) for a in range(Nc) for b in range(a, Nc)]
    else:
        clist = [(a, scid) for a in range(Nc)]
        bA = True
        
    csize = len(clist)
    kernel_dict = dict()
    for cpa, cpb in clist:
        if bA:
            kcpb = cpb
        else:
            kcpb = kk[cpb]
        print(f'______{type(kk)}')
        cpkey = (kk[cpa], kcpb)
        print(f"Processing complexes : {cpkey}")
        kernel_dict[cpkey] = complex_kernel(kk[cpa], kcpb, pdbpklpath, examples, aAll=aA, bAll=bA)
        
    return kernel_dict

def seq_to_dict(examples, kernel_dict, first: bool = True):
    cids = list(set([cid for (cid, _, lbl) in examples]))
    cids.sort()
    dc = dict()
    l0 = 0
    Srn = []   
    for cid in cids:        
        if cid not in dc:
            dc[cid] = [[],[],[],[]]
        if first:           
            if (cid, cid) in kernel_dict:
                tK = kernel_dict[(cid, cid)]            
            else:
                c2 = [c2 for c1, c2 in kernel_dict.keys() if c1 == cid]   
                if len(c2) == 0:
                    continue
                tK = kernel_dict[(cid, c2[0])]
            dc[cid][0] = [tK.lAidx[l] for x, (l, r), lbl in examples if x == cid]
            dc[cid][1] = [tK.rAidx[r] for x, (l, r), lbl in examples if x == cid]
        else:
            c1 = [c1 for c1, c2 in kernel_dict.keys() if c2 == cid]
            tK = kernel_dict[(c1[0], cid)] 
            dc[cid][0] = [tK.lBidx[l] for x, (l, r), lbl in examples if x == cid]
            dc[cid][1] = [tK.rBidx[r] for x, (l, r), lbl in examples if x == cid]            
        dc[cid][2] = [lbl for x, (_, _), lbl in examples if x == cid]
        Srn.extend([(x, (l,r), lbl) for x, (l, r), lbl in examples if x == cid])
        l1 = l0 + len(dc[cid][2])
        dc[cid][3] = range(l0, l1)
        l0 = l1                
    return dc, Srn


def db_kernel_to_pairwise_kernel(kernel_dict, examples, Sc = None, justdiag: bool = False):
    def get_pwk(_cida, _cidb):
        krl = tK.K_rl[dr[_cida][1], :][:, dc[_cidb][0]]
        krr = tK.K_rr[dr[_cida][1], :][:, dc[_cidb][1]]
        kll = tK.K_ll[dr[_cida][0], :][:, dc[_cidb][0]]
        klr = tK.K_lr[dr[_cida][0], :][:, dc[_cidb][1]]
        k_tppk = kll * krr + klr * krl
        return k_tppk
    
    dr, examples = seq_to_dict(examples, kernel_dict)   
    sym: bool = False
    if Sc is None:
        Sc = examples
        dc = dr
        sym = True
    else:
        dc, Sc = seq_to_dict(Sc, kernel_dict, first=False) 
    if justdiag:
        assert sym
        Krc = np.zeros(len(examples), np.single)    
    else:
        Krc = np.zeros((len(examples), len(Sc)), np.single)
    st: int = 0    
    for aidx, cida in enumerate(dr.keys()):    
        if justdiag:
            print(cida)
            tK = kernel_dict[(cida, cida)]
            kab = get_pwk(cida, cida).diagonal()
            Krc[dr[cida][3]] = np.single(kab)
            continue
        if sym:
            st = aidx
        for cidb in list(dc.keys())[st:]:    
            try:
                tK = kernel_dict[(cida, cidb)]
                _cida, _cidb = cida, cidb
            except Exception as exc:
                if sym:
                    _cidb, _cida = cida,cidb
                    tK = kernel_dict[(_cida, _cidb)]
                else:
                    raise exc
            print(_cida)
            print(_cidb)
            kab = get_pwk(_cida, _cidb)
            if _cida != cida:
                kab = kab.T
            Krc[np.ix_(dr[cida][3], dc[cidb][3])] = kab
            if sym:
                Krc[np.ix_(dc[cidb][3], dr[cida][3])] = kab.T                  
    return Krc, examples, Sc

In [20]:
class complex_kernel:
    def __init__(self, cidA, cidB, pdbpklpath, E, aAll = 0, bAll = 0):
        self.cidA = cidA
        self.cidB = cidB
        lAname = os.path.join(pdbpklpath, cidA + '_l_u.pdb.pkl')
        rAname = os.path.join(pdbpklpath, cidA + '_r_u.pdb.pkl')
        lBname = os.path.join(pdbpklpath, cidB + '_l_u.pdb.pkl')
        rBname = os.path.join(pdbpklpath, cidB + '_r_u.pdb.pkl')        
        lA = MyPDB.loader(lAname)
        rA = MyPDB.loader(rAname)
        lB = MyPDB.loader(lBname)
        rB = MyPDB.loader(rBname)
        
        if aAll:
            lAidx = range(len(lA.R))
            rAidx = range(len(rA.R))
        else:
            print(len(tuple(map(np.unique, zip(*E.pos_ex[cidA][0])))))
            print(len(tuple(map(np.unique, zip(*E.neg_ex[cidA])))))
            (lAidx_p, rAidx_p) = tuple(map(np.unique, zip(*E.pos_ex[cidA][0])))
            (lAidx_n, rAidx_n) = tuple(map(np.unique, zip(*E.neg_ex[cidA])))
            lAidx = list(np.unique(np.concatenate((lAidx_p, lAidx_n))))
            rAidx = list(np.unique(np.concatenate((rAidx_p, rAidx_n))))
        if bAll:
            lBidx = range(len(lB.R))
            rBidx = range(len(rB.R))
        else:
            print(len(tuple(map(np.unique, zip(*E.pos_ex[cidB][0])))))
            print(len(tuple(map(np.unique, zip(*E.neg_ex[cidB])))))
            (lBidx_p, rBidx_p) = tuple(map(np.unique, zip(*E.pos_ex[cidB][0])))
            (lBidx_n, rBidx_n) = tuple(map(np.unique, zip(*E.neg_ex[cidB])))
            lBidx = list(np.unique(np.concatenate((lBidx_p, lBidx_n))))
            rBidx = list(np.unique(np.concatenate((rBidx_p, rBidx_n))))
            
        self.K_ll = ProteinKernel(lA, lB, lAidx, lBidx).K
        self.K_lr = ProteinKernel(lA, rB, lAidx, rBidx).K
        self.K_rl = ProteinKernel(rA, lB, rAidx, lBidx).K
        self.K_rr = ProteinKernel(rA, rB, rAidx, rBidx).K
        
        print(f"nans in kernels: {np.sum(np.isnan(self.K_ll))}, {np.sum(np.isnan(self.K_lr))}," + \
              f"{np.sum(np.isnan(self.K_rl))}, {np.sum(np.isnan(self.K_rr))}")
        
        self.lAidx = dict(zip(lAidx, range(len(lAidx))))
        self.rAidx = dict(zip(rAidx, range(len(rAidx))))
        self.lBidx = dict(zip(lBidx, range(len(lBidx))))
        self.rBidx = dict(zip(rBidx, range(len(rBidx))))
        
    def __str__(self):
        return f"complex_kernel object over complexes : {self.cidA}, {self.cidB}"

In [210]:
pdb_path = './targets/'
profile_path = pdb_path + 'profile/'
pdbpklpath = './pdbpkl/'

cidx = set()
files = glob.glob("./targets/" + '*.pdb')
files = glob.glob(pdb_path + '*.pdb') + glob.glob(pdb_path + '*.pdb')
for file in files:
    fl = False
    for b in banned:
        if b in file:
            fl = True
            break
    if fl:
        fl = False
        continue
    cidx.add(file[-12:-8])
incids = list(cidx)

In [211]:
exfname = pdbpklpath + 'EPN6_200.lbl.pkl'
ofname = 'dbdKernel.dbk.pkl'

E = GetExamples.loader(exfname)
dk = db_kernel(pdbpklpath, E, ofname, incids)

Number of Examples (+,-): (4953,7392
Evaluating kernel over: ['1FQJ', '1FFW', '1OPH', '1CLV', '1GLA', '1E96', '1JIW', '1AVX', '1GL1', '2OUL', '1KTZ', '1R0R', '1GPW', '1PXV', '1AY7', '1R6Q', '1IBR', '1GRN', '1EAW', '1LFD', '1F6M', '1BKD', '1B6C', '1H1V', '1JTG', '1OYV', '1FQ1', '1PVH', '1H9D', '1EFN', '1HE1', '1IRA', '1BVN', '1PPE', '1BUH', '1R8S', '1QA9', '1D6R', '1CGI', '1I2M', '1AK4', '1FC2', '1HE8', '1KXP', '1JTD', '1KXQ', '1ACB', '1S1Q', '1J2J', '1GHQ', '1KAC', '1FLE', '1ATN', '1DFJ']
______<class 'list'>
Processing complexes : ('1FQJ', '1FQJ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FQJ', '1FFW')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FQJ', '1OPH')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FQJ', '1CLV')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FQJ', '1GLA')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Proce

2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1HE1')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1IRA')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1BVN')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1PPE')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1BUH')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1R8S')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1QA9')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1D6R')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1CGI')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1FFW', '1I2M')
2
2
2
2
nans in kernels: 0, 0,0, 0
_____

2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1AY7')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1R6Q')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1IBR')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1GRN')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1EAW')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1LFD')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1F6M')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1BKD')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1B6C')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1CLV', '1H1V')
2
2
2
2
nans in kernels: 0, 0,0, 0
_____

2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GLA', '1KAC')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GLA', '1FLE')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GLA', '1ATN')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GLA', '1DFJ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1E96', '1E96')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1E96', '1JIW')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1E96', '1AVX')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1E96', '1GL1')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1E96', '2OUL')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1E96', '1KTZ')
2
2
2
2
nans in kernels: 0, 0,0, 0
_____

2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1AK4')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1FC2')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1HE8')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1KXP')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1JTD')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1KXQ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1ACB')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1S1Q')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1J2J')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1JIW', '1GHQ')
2
2
2
2
nans in kernels: 0, 0,0, 0
_____

...

2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1BUH', '1DFJ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1R8S')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1QA9')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1D6R')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1CGI')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1I2M')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1AK4')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1FC2')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1HE8')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1R8S', '1KXP')
2
2
2
2
nans in kernels: 0, 0,0, 0
_____

2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1AK4')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1FC2')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1HE8')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1KXP')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1JTD')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1KXQ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1ACB')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1S1Q')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1J2J')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1AK4', '1GHQ')
2
2
2
2
nans in kernels: 0, 0,0, 0
_____

nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1J2J', '1FLE')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1J2J', '1ATN')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1J2J', '1DFJ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GHQ', '1GHQ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GHQ', '1KAC')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GHQ', '1FLE')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GHQ', '1ATN')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1GHQ', '1DFJ')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1KAC', '1KAC')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 'list'>
Processing complexes : ('1KAC', '1FLE')
2
2
2
2
nans in kernels: 0, 0,0, 0
______<class 

1E96
1KAC
1E96
1KTZ
1E96
1KXP
1E96
1KXQ
1E96
1LFD
1OPH
1E96
1E96
1OYV
1E96
1PPE
1E96
1PVH
1E96
1PXV
1E96
1QA9
1E96
1R0R
1E96
1R6Q
1E96
1R8S
1E96
1S1Q
1E96
2OUL
1EAW
1EAW
1EAW
1EFN
1EAW
1F6M
1EAW
1FC2
1FFW
1EAW
1EAW
1FLE
1EAW
1FQ1
1FQJ
1EAW
1EAW
1GHQ
1GL1
1EAW
1GLA
1EAW
1GPW
1EAW
1GRN
1EAW
1EAW
1H1V
1EAW
1H9D
1EAW
1HE1
1EAW
1HE8
1EAW
1I2M
1IBR
1EAW
1EAW
1IRA
1EAW
1J2J
1JIW
1EAW
1EAW
1JTD
1EAW
1JTG
1EAW
1KAC
1KTZ
1EAW
1EAW
1KXP
1EAW
1KXQ
1EAW
1LFD
1OPH
1EAW
1EAW
1OYV
1EAW
1PPE
1EAW
1PVH
1PXV
1EAW
1EAW
1QA9
1R0R
1EAW
1R6Q
1EAW
1EAW
1R8S
1EAW
1S1Q
2OUL
1EAW
1EFN
1EFN
1F6M
1EFN
1EFN
1FC2
1FFW
1EFN
1EFN
1FLE
1FQ1
1EFN
1FQJ
1EFN
1EFN
1GHQ
1GL1
1EFN
1GLA
1EFN
1GPW
1EFN
1GRN
1EFN
1H1V
1EFN
1H9D
1EFN
1EFN
1HE1
1EFN
1HE8
1EFN
1I2M
1IBR
1EFN
1EFN
1IRA
1EFN
1J2J
1JIW
1EFN
1EFN
1JTD
1JTG
1EFN
1EFN
1KAC
1KTZ
1EFN
1EFN
1KXP
1EFN
1KXQ
1LFD
1EFN
1OPH
1EFN
1OYV
1EFN
1EFN
1PPE
1PVH
1EFN
1PXV
1EFN
1EFN
1QA9
1R0R
1EFN
1R6Q
1EFN
1EFN
1R8S
1EFN
1S1Q
2OUL
1EFN
1F6M
1F6M
1F6M
1FC2
1FFW
1F6M
1F6M
1FLE
1F6M
1FQ1


1R8S
1R8S
1R8S
1S1Q
2OUL
1R8S
1S1Q
1S1Q
2OUL
1S1Q
2OUL
2OUL
TIME TAKEN FOR KERNEL EVALUATION (s): 364.23972511291504
File Saved: dbdKernel.dbk.pkl
TIME TAKEN FOR Saving kernel file (s): 0.6393032073974609


In [27]:
a = db_kernel.loader("dbdKernel.dbk.pkl")
len(a)

4

In [28]:
a, b, c, d = a

In [32]:
X = a
y = np.array(b[:, 2], dtype=np.int32)

In [33]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler().fit(X)
X = scaler.transform(X)
y = (y + 1) / 2
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, shuffle=True)

In [52]:
from sklearn.model_selection import GridSearchCV

gcv = GridSearchCV(SVC(), param_grid={'kernel': ['linear'], 'C': np.linspace(1, 10, 10)}, n_jobs=30)
gcv.fit(X_train, y_train)

GridSearchCV(estimator=SVC(), n_jobs=30,
             param_grid={'C': array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.]),
                         'kernel': ['linear']})

In [38]:
d.loader('./pdbpkl/EPN6_200.lbl.pkl').pos_ex

{'1KXQ': ([(50, 50),
   (70, 51),
   (71, 51),
   (26, 52),
   (49, 52),
   (68, 52),
   (69, 52),
   (70, 52),
   (71, 52),
   (72, 52),
   (73, 52),
   (74, 52),
   (26, 53),
   (49, 53),
   (73, 53),
   (50, 57),
   (26, 58),
   (28, 58),
   (49, 58),
   (50, 58),
   (51, 58),
   (50, 62),
   (51, 62),
   (51, 102),
   (50, 103),
   (51, 103),
   (51, 104),
   (52, 104),
   (53, 104),
   (51, 105),
   (52, 105),
   (53, 105),
   (70, 105),
   (51, 106),
   (70, 107),
   (61, 143),
   (53, 144),
   (54, 144),
   (55, 144),
   (56, 144),
   (53, 145),
   (53, 146),
   (53, 147),
   (98, 147),
   (99, 147),
   (100, 147),
   (101, 147),
   (53, 148),
   (55, 148),
   (99, 148),
   (101, 148),
   (102, 148),
   (101, 149),
   (98, 150),
   (99, 150),
   (100, 150),
   (53, 160),
   (51, 161),
   (98, 161),
   (48, 162),
   (50, 162),
   (51, 162),
   (52, 162),
   (53, 162),
   (97, 162),
   (98, 162),
   (99, 162),
   (100, 162),
   (51, 163),
   (53, 163),
   (51, 164),
   (53, 164),


In [53]:
import pandas as pd

pd.DataFrame(gcv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2135.762180,213.601223,467.525529,82.492585,1.0,linear,"{'C': 1.0, 'kernel': 'linear'}",0.689994,0.696181,0.711227,0.693287,0.719907,0.702119,0.011486,10
1,2103.594633,239.235873,540.632186,129.621720,2.0,linear,"{'C': 2.0, 'kernel': 'linear'}",0.700983,0.702546,0.717593,0.700810,0.714699,0.707326,0.007284,9
2,2194.745135,339.945568,600.179011,101.292025,3.0,linear,"{'C': 3.0, 'kernel': 'linear'}",0.702718,0.696759,0.722801,0.709491,0.722222,0.710798,0.010379,8
3,2377.204627,60.031541,392.338002,24.251590,4.0,linear,"{'C': 4.0, 'kernel': 'linear'}",0.704453,0.697917,0.722801,0.714699,0.726852,0.713344,0.010870,7
4,2311.572556,181.911131,458.744000,122.637582,5.0,linear,"{'C': 5.0, 'kernel': 'linear'}",0.708502,0.701389,0.723958,0.718750,0.728588,0.716237,0.009988,6
5,2383.465188,34.446049,388.573513,11.229118,6.0,linear,"{'C': 6.0, 'kernel': 'linear'}",0.713707,0.703704,0.719907,0.721644,0.732639,0.718320,0.009524,5
6,1768.199547,87.589588,75.855319,29.703232,7.0,linear,"{'C': 7.0, 'kernel': 'linear'}",0.716021,0.707755,0.721644,0.724537,0.733218,0.720635,0.008506,1
7,1623.347450,151.905355,81.284766,21.384959,8.0,linear,"{'C': 8.0, 'kernel': 'linear'}",0.717756,0.704861,0.718750,0.725116,0.731481,0.719593,0.008865,4
8,1416.863480,128.097711,102.327669,17.225342,9.0,linear,"{'C': 9.0, 'kernel': 'linear'}",0.715442,0.710648,0.718171,0.722222,0.731481,0.719593,0.007036,3
9,1341.951139,36.551633,96.666615,16.533394,10.0,linear,"{'C': 10.0, 'kernel': 'linear'}",0.716599,0.713542,0.718750,0.721644,0.729167,0.719940,0.005321,2


In [29]:
b = [[*f] for f in b]

In [30]:
b = np.array(b)

<ipython-input-30-be524582d8b4>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  b = np.array(b)


## Generating data folds

In [ ]:
a, b, c, d = db_kernel.loader("dbdKernel.dbk.pkl")
b = [[*f] for f in b]
b = np.array(b)

In [64]:
pattern_ids = list(zip(*list(zip(*b))[:2]))
y = np.array(b[:, 2], dtype=np.int32)
labels = y

X = a
scaler = MinMaxScaler().fit(X)
X = scaler.transform(X)

In [94]:
import random

def chunks(l, n):
    for i in range(0, len(l), n):
        yield l[i: i + n]

dkey = dict()
N_FOLDS = 5

dkey = dict()
for i, e in enumerate(pattern_ids):
    if e[0] not in dkey.keys():
        dkey[e[0]] = [[], []]
    dkey[e[0]][0].append(i)
    dkey[e[0]][1].append(e)
    
cids = list(dkey.keys())
random.shuffle(cids)
fcids = list(chunks(cids, int(np.ceil(len(cids) / float(N_FOLDS)))))
F = [[] for _ in range(N__FOLDS)]
for f in range(N__FOLDS):
    for cid in fcids[f]:
        if cid in dkey:
            F[f].extend(dkey[cid][0])
        else:
            print(f'Warning: Complex {cid} not found in the kernel. We skip it', file=sys.stderr)

In [95]:
class CustomCVFold:
    def __init__(self, n_splits=5):
        self.n_splits = n_splits
        self.X_train = []
        self.X_test = F
        L = set(itertools.chain(*self.X_test))
        for f in range(n_splits):
            self.X_train.append(list(L.difference(self.X_test[f])))

    def split(self, X, y, groups=None):
        for i in range(self.n_splits):
            yield self.X_train[i], self.X_test[i]

    def get_n_splits(self, X, y, groups=None):
        return self.n_splits

In [106]:
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import roc_auc_score

gcv = GridSearchCV(SVC(),
                   param_grid={'kernel': ['precomputed'], 'C': np.linspace(1, 3, 1)},
                   n_jobs=20, cv=CustomCVFold(), scoring='roc_auc', verbose=3)
gcv.fit(X, y)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


GridSearchCV(cv=<__main__.CustomCVFold object at 0x7fc50ed037c0>,
             estimator=SVC(), n_jobs=20,
             param_grid={'C': array([1.]), 'kernel': ['precomputed']},
             scoring='roc_auc', verbose=3)

In [107]:
pd.DataFrame(gcv.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_C,param_kernel,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,2.616957,0.248151,0.22351,0.026254,1.0,precomputed,"{'C': 1.0, 'kernel': 'precomputed'}",0.717462,0.713346,0.744839,0.746769,0.732424,0.730968,0.01369,1


In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import RocCurveDisplay
train_idx, test_idx = next(CustomCVFold().split(X, y))
clf = SVC(**gcv.best_params_).fit(X, y)
y_pred = clf.predict_proba(X[test_idx])[:, 1]

print(f'roc_auc_score: {roc_auc_score(y[test_idx], y_pred)}')

fpr, tpr, _ = roc_curve(y[test_idx], y_pred, pos_label=clf.classes_[1])
roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot()

/usr/local/lib/python3.8/dist-packages/sklearn/utils/validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


In [54]:
kernel_dataframe = pd.DataFrame(X, columns=)

array([['1ACB', (46, 38), 1],
       ['1ACB', (47, 38), 1],
       ['1ACB', (48, 38), 1],
       ...,
       ['2OUL', (34, 172), -1],
       ['2OUL', (31, 118), -1],
       ['2OUL', (31, 182), -1]], dtype=object)